In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import signal

In [195]:
test_data = pd.read_csv('data/Texas/TexasTests.csv')
data = pd.read_csv('data/Texas/TexasData.csv')

In [196]:
def date_address(date):
    dates = date.split("/")
    first = dates[0]
    if len(first) == 4:
        dates[0] = first[2:]
    dates = ["20"+dates[2], dates[0], dates[1]]
    return "/".join(dates)

data["Date"] = data["Date"].map(date_address)
data.head(25)

test_data["Date"] = test_data["Date"].map(date_address)
test_data.head(25)

,Date,Anderson,Andrews,Angelina,Aransas,Archer,Armstrong,Atascosa,Austin,Bailey,...,Willacy,Williamson,Wilson,Winkler,Wise,Wood,Yoakum,Young,Zapata,Zavala
0,2020/4/22,12,0,29,1,1,1,3,2,0,...,3,128,0,0,7,6,0,11,0,0
1,2020/4/23,15,3,120,-2,4,3,-8,-89,1,...,49,-473,14,3,-48,-27,-12,-226,3,3
2,2020/4/24,32,1,3,9,5,0,24,138,0,...,18,138,83,2,57,27,9,173,2,2
3,2020/4/25,8,0,35,3,2,0,5,19,0,...,11,207,-43,2,28,3,1,10,1,-2
4,2020/4/26,0,0,-1,0,0,0,1,0,0,...,0,53,6,0,4,0,0,0,0,0
5,2020/4/27,23,0,76,0,0,2,3,10,0,...,9,33,5,0,17,2,0,7,3,0
6,2020/4/28,0,0,8,0,0,1,2,5,0,...,2,11,3,0,2,0,0,0,1,0
7,2020/4/29,37,0,74,1,0,0,5,6,0,...,4,52,8,0,24,4,0,0,0,0
8,2020/4/30,19,0,41,1,1,0,7,3,1,...,1,50,3,0,6,1,0,2,1,1
9,2020/5/1,-1,0,-1,0,0,0,0,0,0,...,0,-1,1,0,0,0,0,0,0,0


In [197]:
data =data.T
new_column =data.iloc[0].values
data.columns = new_column
data = data[1:]

test_data =test_data.T
new_column =test_data.iloc[0].values
test_data.columns = new_column
test_data = test_data[1:]
test_data.head()

,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26,2020/4/27,2020/4/28,2020/4/29,2020/4/30,2020/5/1,...,2021/11/11,2021/11/12,2021/11/13,2021/11/14,2021/11/15,2021/11/16,2021/11/17,2021/11/18,2021/11/19,2021/11/20
Anderson,12,15,32,8,0,23,0,37,19,-1,...,114,136,253,162,208,195,59,60,88,12
Andrews,0,3,1,0,0,0,0,0,0,0,...,13,10,2,4,25,12,10,12,11,1
Angelina,29,120,3,35,-1,76,8,74,41,-1,...,151,54,528,134,462,140,49,223,395,27
Aransas,1,-2,9,3,0,0,0,1,1,0,...,24,31,39,49,77,59,44,37,41,9
Archer,1,4,5,2,0,0,0,0,1,0,...,17,22,17,12,22,9,7,6,21,0


In [198]:
data.reset_index(inplace=True)
data = data.rename(columns = {'index':'County'})
#data.head()

test_data.reset_index(inplace=True)
test_data = test_data.rename(columns = {'index':'County'})
test_data.head()

,County,2020/4/22,2020/4/23,2020/4/24,2020/4/25,2020/4/26,2020/4/27,2020/4/28,2020/4/29,2020/4/30,...,2021/11/11,2021/11/12,2021/11/13,2021/11/14,2021/11/15,2021/11/16,2021/11/17,2021/11/18,2021/11/19,2021/11/20
0,Anderson,12,15,32,8,0,23,0,37,19,...,114,136,253,162,208,195,59,60,88,12
1,Andrews,0,3,1,0,0,0,0,0,0,...,13,10,2,4,25,12,10,12,11,1
2,Angelina,29,120,3,35,-1,76,8,74,41,...,151,54,528,134,462,140,49,223,395,27
3,Aransas,1,-2,9,3,0,0,0,1,1,...,24,31,39,49,77,59,44,37,41,9
4,Archer,1,4,5,2,0,0,0,0,1,...,17,22,17,12,22,9,7,6,21,0


In [199]:
data = data.melt(
    id_vars = ["County"],
    var_name = "Date",
    value_name = "Confirmed Cases")

test_data = test_data.melt(
    id_vars = ["County"],
    var_name = "Date",
    value_name = "Confirmed Cases")

In [200]:
test_data

,County,Date,Confirmed Cases
0,Anderson,2020/4/22,12
1,Andrews,2020/4/22,0
2,Angelina,2020/4/22,29
3,Aransas,2020/4/22,1
4,Archer,2020/4/22,1
...,...,...,...
146807,Wood,2021/11/20,4
146808,Yoakum,2021/11/20,1
146809,Young,2021/11/20,1
146810,Zapata,2021/11/20,3


In [184]:
data.to_csv("TexasDataVisualize.csv", index = False)

In [201]:
test_data.to_csv("TexasTestsVisualize.csv", index = False)